# Exploring Hacker News Posts

## Introduction

In this project, we'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. These different posts are `Ask HN` and `Show HN`.

Users submit `Ask HN` posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

We will be comparing these posts to explore the following:
* Which one of the two kinds of posts receive more comments (on average)?
* Does time impact the average number of comments received?

The full data set can be seen [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts) but it was reduced to ~20,000 rows, as many of the submissions received no comments and thus these submissions are not pertinent to our exploration. We then took a random sample of the remaining submissions to arrive at our final data set.

# The Data

In [3]:
# Opens file and displays first five rows/headers as a test
from csv import reader
opened_file=open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn=hn[1:]

print(hn[:5])
print(headers)

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


The data contains information on the submission id, submission title, the post url, the number of points and comments the submission received, as well as who created the submission and when it was posted.

We will start the exploration by separating posts into different lists.

In [4]:
# Gives a count for the number of ask, show, and other posts in the dataset
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts), len(show_posts), len(other_posts))

1744 1162 17194


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [5]:
# Find the total and average comments for both ask and show comments
total_ask_comments = 0
for post in ask_posts:
    comment = post[4]
    comment = int(comment)
    total_ask_comments += comment
average_ask_comments = total_ask_comments/len(ask_posts)
print(average_ask_comments)

total_show_comments=0
for post in show_posts:
    comment = post[4]
    comment = int(comment)
    total_show_comments += comment
average_show_comments = total_show_comments/len(show_posts)
print(average_show_comments)

14.038417431192661
10.31669535283993


On average, there are about 3.7 more ask comments than show comments. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

# Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [6]:
# Uses datetime to find the number of comments per hour (comments_per_hour) in ask posts
# Also finds the number of posts per hour (counts_by_hour) in ask posts
import datetime as dt

result_list = []
for post in ask_posts:
    created_at = post[6]
    comment = post[4]
    comment=int(comment)
    inner_list=[created_at, comment]
    result_list.append(inner_list)

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date=row[0]
    comment = row[1]
    date_dt = dt.datetime.strptime(date, "%m/%d/%Y %H:%M").strftime("%H")
    if date_dt in counts_by_hour:
        counts_by_hour[date_dt] += 1
        comments_by_hour[date_dt] += comment
    else:
        counts_by_hour[date_dt] = 1
        comments_by_hour[date_dt] = comment
print(counts_by_hour, comments_by_hour)
    
    

{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58} {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


# Calculating the Average Number of Comments for Ask HN Posts by Hour


In [7]:
avg_per_hour = []
for hour in comments_by_hour:
    avg_per_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])
print(avg_per_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


In [8]:
swap_avg_by_hour = []
for row in avg_per_hour:
    swap_avg_by_hour.append([row[1], row[0]])
sorted_swap = sorted(swap_avg_by_hour, key=None, reverse=True)


for avg, hour in sorted_swap[0:5]:
    print("{}: {:.2f} average comments per post".format(dt.datetime.strptime(hour, "%H").strftime("%H:%M"), avg))
    

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).